# Basic Stock Analysis

How to evaluate the performance of your assets?
Optimize your portfolio by reviewing the statistics of your stocks.

### 1. Importing Libraries

In [1]:
import pandas_datareader as pdr
import numpy as np
import datetime as dt
import pandas as pd
from sklearn.linear_model import LinearRegression

### 2. Calculating Beta
Beta expresses how "dynamic" an asset reacts to market fluctuations. A high beta means high risk, but also high opportunity, depending on the market.
- Beta < 1: More stable, but also less growth opportunity
- Beta > 1: More risky, but also more gain opportunity

#### Method
- Get raw stock data of the market and the stocks of interest (can be 1 year, 3 years or other timeframe). Typically the S&P500 (Symbol "^GSPC") is used as the representation of the market data.
- Transform raw data into a dataframe with returns
- Calculate Beta
- Calculate the expected rate of return (CAPM method)
- Interprete the results

In [10]:
tickers = ['XIC.TO', '^GSPC', 'SHOP', 'TSLA', 'VFV.TO']
start = dt.datetime(2018, 12, 24)
end = dt.datetime(2021, 12, 24)
df_stocks = pdr.get_data_yahoo(tickers, start, end, interval="m")

df_stocks = df_stocks['Adj Close']

df_stocks

Symbols,XIC.TO,^GSPC,SHOP,TSLA,VFV.TO
Date,,,,,
2019-01-01,22.738510,2704.100098,168.470001,61.403999,60.665508
2019-02-01,23.454830,2784.489990,189.149994,63.976002,62.673283
2019-03-01,23.519115,2834.399902,206.619995,55.972000,64.613815
2019-04-01,24.436722,2945.830078,243.529999,47.737999,67.675255
2019-05-01,23.715271,2752.060059,274.899994,37.032001,63.891380
2019-06-01,24.085249,2941.760010,300.149994,44.692001,65.870186
2019-07-01,24.377037,2980.379883,317.880005,48.321999,67.781433
2019-08-01,24.488943,2926.459961,385.390015,45.122002,67.190102
2019-09-01,24.703432,2976.739990,311.660004,48.174000,67.839600


### Calculating Beta - Method 1

In [11]:
# Generating dataframe with returns
df_returns = np.log(df_stocks / df_stocks.shift())
df_returns

Symbols,XIC.TO,^GSPC,SHOP,TSLA,VFV.TO
Date,,,,,
2019-01-01,NaN,NaN,NaN,NaN,NaN
2019-02-01,0.031016,0.029296,0.115783,0.041033,0.032560
2019-03-01,0.002737,0.017766,0.088341,-0.133656,0.030493
2019-04-01,0.038273,0.038560,0.164359,-0.159124,0.046292
2019-05-01,-0.029968,-0.068041,0.121167,-0.253945,-0.057536
2019-06-01,0.015480,0.066658,0.087875,0.188012,0.030501
2019-07-01,0.012042,0.013043,0.057392,0.078092,0.028602
2019-08-01,0.004580,-0.018257,0.192582,-0.068517,-0.008762
2019-09-01,0.008720,0.017035,-0.212343,0.065449,0.009620


In [12]:
cov = df_returns.cov()
var = df_returns['^GSPC'].var()
beta_xic = cov.loc['XIC.TO', '^GSPC'] / var
beta_shop = cov.loc['SHOP', '^GSPC'] / var
beta_tsla = cov.loc['TSLA', '^GSPC'] / var
beta_vfv = cov.loc['VFV.TO', '^GSPC'] / var

In [13]:
print('Betas for XIC: {:1.2}, SHOPIFY: {:1.2}, Tesla: {:1.2}, Vanguard: {:1.2}'.format(beta_xic, beta_shop, beta_tsla, beta_vfv))

Betas for XIC: 0.93, SHOPIFY: 1.1, Tesla: 2.4, Vanguard: 0.79


### Calculating Beta - Method 2

In [8]:
def calc_return(df):
    '''Function to generate a detaframe with returns
    - Input: Dataframe with raw stock data
    - Output: Dataframe with returns'''

    x = df.copy()
    
    for i in range(1, len(df)):
        x.iloc[i] = (df.iloc[i] - df.iloc[i-1]) / df.iloc[i-1] * 100

    x.iloc[0] = 0

    return x

In [ ]:
df_returns = 